In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [45]:
df_train=pd.read_csv("E:\\jinnan\\train_after_clean.csv")
df_train=df_train[df_train['yield_rate']>0.87]
df_train.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)
for col in df_train.columns:
    rate = df_train[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        df_train.drop(columns=[col], axis=1, inplace=True)
        
df_test=pd.read_csv("E:\\jinnan\\test_after_clean.csv")
df_test.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)
for col in df_test.columns:
    rate = df_test[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        df_test.drop(columns=[col], axis=1, inplace=True)
print(df_train.shape,df_test.shape)

(1381, 40) (150, 39)


In [46]:
target=df_train['yield_rate']
df_train.drop(columns=['yield_rate'],axis=1,inplace=True)
data=pd.concat([df_train,df_test],axis=0)
data["numerical_id"]=data['id'].apply(lambda x: int(x[7:]))
data.drop(columns=['id'],axis=1,inplace=True)
print(data.columns,data.shape)

Index(['A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'A19', 'A20',
       'A20range', 'A21', 'A22', 'A24', 'A25', 'A26', 'A27', 'A28', 'A28range',
       'A5', 'A6', 'A7', 'A8', 'A9', 'B1', 'B10', 'B10range', 'B11',
       'B11range', 'B12', 'B14', 'B4', 'B4range', 'B5', 'B6', 'B7', 'B8', 'B9',
       'B9range', 'numerical_id'],
      dtype='object') (1531, 39)


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  This is separate from the ipykernel package so we can avoid doing imports until


In [47]:
col_list=data.columns
for col,index in zip(col_list,range(data.shape[1])):
    for _col in col_list[index:]:
        data[col+"multiply"+_col]=data[col]*data[_col]
        print(col,'*',_col)
#         data[col+"divide"+_col]=data[]

A10 * A10
A10 * A11
A10 * A12
A10 * A14
A10 * A15
A10 * A16
A10 * A17
A10 * A19
A10 * A20
A10 * A20range
A10 * A21
A10 * A22
A10 * A24
A10 * A25
A10 * A26
A10 * A27
A10 * A28
A10 * A28range
A10 * A5
A10 * A6
A10 * A7
A10 * A8
A10 * A9
A10 * B1
A10 * B10
A10 * B10range
A10 * B11
A10 * B11range
A10 * B12
A10 * B14
A10 * B4
A10 * B4range
A10 * B5
A10 * B6
A10 * B7
A10 * B8
A10 * B9
A10 * B9range
A10 * numerical_id
A11 * A11
A11 * A12
A11 * A14
A11 * A15
A11 * A16
A11 * A17
A11 * A19
A11 * A20
A11 * A20range
A11 * A21
A11 * A22
A11 * A24
A11 * A25
A11 * A26
A11 * A27
A11 * A28
A11 * A28range
A11 * A5
A11 * A6
A11 * A7
A11 * A8
A11 * A9
A11 * B1
A11 * B10
A11 * B10range
A11 * B11
A11 * B11range
A11 * B12
A11 * B14
A11 * B4
A11 * B4range
A11 * B5
A11 * B6
A11 * B7
A11 * B8
A11 * B9
A11 * B9range
A11 * numerical_id
A12 * A12
A12 * A14
A12 * A15
A12 * A16
A12 * A17
A12 * A19
A12 * A20
A12 * A20range
A12 * A21
A12 * A22
A12 * A24
A12 * A25
A12 * A26
A12 * A27
A12 * A28
A12 * A28range
A12 * A5
A

In [48]:
data_collist=list(data.columns)
print(len(data_collist))
for col in col_list:
    data_collist.remove(col)
print(len(data_collist))

819
780


In [49]:
print(data.shape,df_train.shape,df_test.shape)
df_train=data[data_collist][:df_train.shape[0]]
df_test=data[data_collist][df_train.shape[0]:]
print(data.shape,df_train.shape,df_test.shape)
# df_train.to_csv("E:\\jinnan\\multiply_feature_train.csv",index=False)
# df_test.to_csv("E:\\jinnan\\multiply_feature_test.csv",index=False)

(1531, 819) (1381, 39) (150, 39)
(1531, 819) (1381, 780) (150, 780)


In [2]:
from sklearn import preprocessing
from sklearn.model_selection import KFold, RepeatedKFold
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [3]:
selected_col=['A12multiplyB14', 'B14_to_B7_intTarget_4.0_mean', 'A15multiplyB14', 'A10multiplyB14', 'B12multiplyB14',
              'A17multiplyB14', 'B11rangemultiplynumerical_id', 'A27multiplyB14', 'B11rangemultiplyB14', 'A14multiplyB14', 
              'A11', 'A10multiplyA9', 'A20rangemultiplyB14', 'A9multiplynumerical_id', 'A14', 'A14multiplyA6',
              'A20multiplyB14', 'A25multiplyB14', 'A9', 'B14', 'B10multiplynumerical_id',
              'A10', 'A5multiplyB14', 'B14_to_A5_intTarget_0.0_mean', 'A5multiplynumerical_id', 'A12multiplyA15',
              'A10multiplyB11range', 'A11multiplynumerical_id', 'B1multiplyB14', 'id', 'B10multiplyB6',
              'B6multiplyB7', 'A15', 'A27multiplyB6', 'A24', 'A22multiplyB6',
              'id_dispersion', 'B6', 'B14multiplyB6', 'A12', 'A19multiplyB14', 
              'A10multiplyB11', 'A10multiplynumerical_id', 'B11multiplyB14', 'A11multiplyA6', 'B6multiplynumerical_id',
              'A6multiplynumerical_id', 'B14multiplynumerical_id', 'A17multiplyA19', 'B10rangemultiplyB6', 'B9rangemultiplynumerical_id', 
              'B7', 'A12multiplyA17', 'A10multiplyA25', 'A20rangemultiplynumerical_id', 'A6multiplyB10', 
              'A20', 'A27multiplynumerical_id', 'B10multiplyB14', 'A25multiplynumerical_id', 'A14multiplyB7', 
              'A5multiplyA6', 'A22', 'B10range', 'A20rangemultiplyA6', 'A26multiplynumerical_id',
              'B10rangemultiplynumerical_id', 'A21', 'A28range', 'A6multiplyB10range', 'A15multiplyA19', 
              'B1multiplynumerical_id', 'A19multiplynumerical_id', 'B7multiplyB9range', 'A28rangemultiplynumerical_id', 'A15multiplyA17',
              'A25multiplyB7', 'B11multiplynumerical_id', 'A17', 'A21multiplyA6', 'B9multiplynumerical_id',
              'A12multiplyA20range', 'B7multiplynumerical_id', 'A6multiplyB9range', 'B14multiplyB7', 'B9',
              'B4rangemultiplynumerical_id', 'A17multiplyB6', 'A6multiplyB14', 'A5multiplyB6', 'A19multiplyB6',
              'A10multiplyB6', 'A20rangemultiplyA27', 'A28multiplynumerical_id', 'A14multiplynumerical_id', 'A19multiplyA6', 
              'A12multiplyA25', 'A11multiplyA28range', 'A5multiplyB10', 'A10multiplyB10', 'A6',
              'A25multiplyB11', 'B8', 'B4multiplynumerical_id', 'B5multiplynumerical_id', 'A27multiplyA5',
              'A25', 'A6multiplyB6', 'A10multiplyB10range', 'A6multiplyB9', 'B1multiplyB6', 
              'A15multiplynumerical_id', 'A12multiplyB7', 'A21multiplynumerical_id', 'A6multiplyB1', 'A25multiplyB6', 
              'A17multiplyA25', 'B11', 'A5', 'B10', 'A20multiplynumerical_id',
              'A24multiplynumerical_id', 'A10multiplyA15', 'A10multiplyB7', 'A15multiplyA25', 'B12multiplynumerical_id',
              'A25multiplyA5', 'A11multiplyA5', 'A25multiplyB10', 'A21multiplyA28range', 'A10multiplyA17',
              'A12multiplynumerical_id', 'A16', 'A10multiplyB9range', 'A11multiplyA15', 'A16multiplynumerical_id', 
              'A6multiplyB8', 'A17multiplyA5', 'A11multiplyB6', 'A17multiplyA6', 'A27multiplyA6',
              'A11multiplyB7', 'A17multiplyB10range', 'A17multiplynumerical_id', 'A15multiplyA27', 'A15multiplyA6', 
              'A10multiplyA22', 'A6multiplyB7', 'A21multiplyA27', 'A22multiplyA6', 'A24multiplyA5', 'A25multiplyA6',
              'A25multiplyA27', 'A26', 'A17multiplyA27', 'A12multiplyA19', 'B4rangemultiplyB6', 'B1multiplyB9range',
              'A12multiplyB8', 'A10multiplyA14', 'A25multiplyB1', 'A22multiplynumerical_id', 'A15multiplyA22',
              'A17multiplyA22', 'A5multiplyB7', 'A10multiplyA5', 'B1', 'A15multiplyB6', 'A11multiplyA27', 
              'A12multiplyB6', 'A27multiplyB11', 'A11multiplyB10', 'A12multiplyB10', 'A12multiplyA22',
              'A15multiplyB1', 'A19', 'A10multiplyA19', 'A27multiplyB10', 'A11multiplyA25', 
              'A28rangemultiplyA6', 'A10multiplyA11', 'B1multiplyB4range', 'A27', 'A28',
              'B9range', 'A7multiplynumerical_id', 'A12multiplyA27', 'A10multiplyB1', 'A6_dispersion',
              'A10multiplyA12', 'A12multiplyB9range', 'B1multiplyB8', 'A12multiplyA6', 'B4', 
              'A15multiplyA5', 'A15multiplyA21', 'A15multiplyB4range', 'A12multiplyB4range', 'A11multiplyA17',
              'A20multiplyB10', 'A17multiplyB9range', 'A12multiplyB1', 'A27multiplyB1', 'B8multiplynumerical_id',
              'A11multiplyB14', 'A10multiplyA27', 'A12multiplyA5', 'A17multiplyB4range', 'A6multiplyB4range',
              'A28multiplyA6', 'B5', 'B4rangemultiplyB8', 'A11multiplyA28', 'A20multiplyB4range',
              'A20range', 'A28multiplyB10', 'A20multiplyA6', 'A20multiplyB7', 'A17multiplyB1',
              'A11multiplyA12', 'A10multiplyA6', 'A11multiplyB4range', 'A10multiplyB4range']

In [5]:
df_train=pd.read_csv("E:\\jinnan\\train_dispersion.csv")
df_train=df_train[df_train['yield_rate']>0.87]
df_train.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)
for col in df_train.columns:
    rate = df_train[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        df_train.drop(columns=[col], axis=1, inplace=True)
label_array=df_train["yield_rate"].values
df_train["id"]=df_train['id'].apply(lambda x: int(x[7:]))
df_train.drop(columns=['yield_rate'],axis=1,inplace=True)
print('train1:',df_train.shape)

df_train2=pd.read_csv("E:\\jinnan\\train_binning_mean.csv")
print('train2:',df_train2.shape)
df_train.index=range(df_train.shape[0])
df_train2.index=range(df_train2.shape[0])
df_train=pd.concat([df_train,df_train2],axis=1)

df_train3=pd.read_csv("E:\\jinnan\\multiply_feature_train.csv")
print('train3:',df_train3.shape)
df_train.index=range(df_train.shape[0])
df_train3.index=range(df_train3.shape[0])
df_train=pd.concat([df_train,df_train3],axis=1)

train_array=df_train[selected_col].values
print(df_train.columns,df_train.shape,train_array.shape)

df_test=pd.read_csv("E:\\jinnan\\test_dispersion.csv")
df_test.drop(['B3', 'B13', 'A13', 'A18', 'A23'], axis=1, inplace=True)
for col in df_test.columns:
    rate = df_test[col].value_counts(normalize=True, dropna=False).values[0]
    if rate > 0.9:
        df_test.drop(columns=[col], axis=1, inplace=True)
columns_list=list(df_test.columns)
# columns_list.remove("id")
sample_id=df_test["id"]
df_test["id"]=df_test['id'].apply(lambda x: int(x[7:]))

df_test2=pd.read_csv("E:\\jinnan\\testA_binning_mean.csv")
df_test=pd.concat([df_test,df_test2],axis=1)

df_test3=pd.read_csv("E:\\jinnan\\multiply_feature_test.csv")
df_test=pd.concat([df_test,df_test3],axis=1)

test_array=df_test[selected_col].values
print(df_test.columns,df_test.shape,test_array.shape)

train1: (1381, 78)
train2: (1381, 105)
train3: (1381, 780)
Index(['A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'A19', 'A20',
       'A20range',
       ...
       'B8multiplyB8', 'B8multiplyB9', 'B8multiplyB9range',
       'B8multiplynumerical_id', 'B9multiplyB9', 'B9multiplyB9range',
       'B9multiplynumerical_id', 'B9rangemultiplyB9range',
       'B9rangemultiplynumerical_id', 'numerical_idmultiplynumerical_id'],
      dtype='object', length=963) (1381, 963) (1381, 223)
Index(['A10', 'A11', 'A12', 'A14', 'A15', 'A16', 'A17', 'A19', 'A20',
       'A20range',
       ...
       'B8multiplyB8', 'B8multiplyB9', 'B8multiplyB9range',
       'B8multiplynumerical_id', 'B9multiplyB9', 'B9multiplyB9range',
       'B9multiplynumerical_id', 'B9rangemultiplyB9range',
       'B9rangemultiplynumerical_id', 'numerical_idmultiplynumerical_id'],
      dtype='object', length=963) (150, 963) (150, 223)


In [6]:
param = {'num_leaves': 120,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'mse',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_lgb = np.zeros(train_array.shape[0])
predictions_lgb = np.zeros(test_array.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_array, label_array)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(train_array[trn_idx], label_array[trn_idx])
    val_data = lgb.Dataset(train_array[val_idx], label_array[val_idx])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(train_array[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(test_array, num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.8f}".format(mean_squared_error(oof_lgb, label_array)))

fold n°1
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000152586	valid_1's l2: 0.000179206
[400]	training's l2: 9.5173e-05	valid_1's l2: 0.000129732
[600]	training's l2: 7.54059e-05	valid_1's l2: 0.000119842
[800]	training's l2: 6.4965e-05	valid_1's l2: 0.000116543
[1000]	training's l2: 5.8681e-05	valid_1's l2: 0.000115154
[1200]	training's l2: 5.4511e-05	valid_1's l2: 0.000114375
[1400]	training's l2: 5.13783e-05	valid_1's l2: 0.000114128
[1600]	training's l2: 4.90252e-05	valid_1's l2: 0.000113874
Early stopping, best iteration is:
[1544]	training's l2: 4.96306e-05	valid_1's l2: 0.000113823
fold n°2
Training until validation scores don't improve for 100 rounds.
[200]	training's l2: 0.000153651	valid_1's l2: 0.000183534
[400]	training's l2: 9.25787e-05	valid_1's l2: 0.000150407
[600]	training's l2: 7.20975e-05	valid_1's l2: 0.000144299
[800]	training's l2: 6.1309e-05	valid_1's l2: 0.000142873
[1000]	training's l2: 5.48902e-05	valid_1's l2: 0.0001

In [6]:
df_predicted=pd.DataFrame({"id":sample_id.values,"predicted":predictions_lgb})
df_predicted.loc[:,"predicted"]=df_predicted["predicted"].apply(lambda x: round(x,3))
df_predicted.to_csv("E:\\jinnan\\predict_result\\jinnan_selected_lgb_1_15.csv",header=None,index=False)
df_predicted

,id,predicted
0,sample_1656,0.904
1,sample_1548,0.883
2,sample_769,0.931
3,sample_1881,0.903
4,sample_1807,0.930
5,sample_145,0.922
6,sample_1212,0.936
7,sample_944,0.900
8,sample_829,0.934
9,sample_616,0.925


In [1]:
xgb_params = {'eta': 0.005, 'max_depth': 10, 'subsample': 0.8, 'colsample_bytree': 0.8,
          'objective': 'reg:linear', 'eval_metric': 'rmse', 'silent': True, 'nthread': 4}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_xgb = np.zeros(len(df_train))
predictions_xgb = np.zeros(len(df_test))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_array, label_array)):
    print("fold n°{}".format(fold_ + 1))
    trn_data = xgb.DMatrix(train_array[trn_idx], label_array[trn_idx])
    val_data = xgb.DMatrix(train_array[val_idx], label_array[val_idx])

    watchlist = [(trn_data, 'train'), (val_data, 'valid_data')]
    clf = xgb.train(dtrain=trn_data, num_boost_round=20000, evals=watchlist, early_stopping_rounds=200,
                    verbose_eval=100, params=xgb_params)
    oof_xgb[val_idx] = clf.predict(xgb.DMatrix(train_array[val_idx]), ntree_limit=clf.best_ntree_limit)
    predictions_xgb += clf.predict(xgb.DMatrix(test_array), ntree_limit=clf.best_ntree_limit) / folds.n_splits
    
print("CV score: {:<8.8f}".format(mean_squared_error(oof_xgb, label_array)))

NameError: name 'KFold' is not defined

In [120]:
col_list=df_test.columns
df_impor_sort=pd.DataFrame({"feature":list(df_importance.columns),"importance":list(df_importance.values.T)})
df_impor_sort['feat_index']=df_impor_sort['feature'].apply(lambda x: int(x[1:]))
df_impor_sort['feature']=df_impor_sort['feat_index'].apply(lambda x: col_list[x])
# print(df_impor_sort.sort_values(['importance'],ascending=False))
selected_col_array=df_impor_sort[['feature']][df_impor_sort['importance']>100].values.flatten()

In [125]:
selected_col=selected_col_array.tolist()
print(selected_col)

['A12multiplyB14', 'B14_to_B7_intTarget_4.0_mean', 'A15multiplyB14', 'A10multiplyB14', 'B12multiplyB14', 'A17multiplyB14', 'B11rangemultiplynumerical_id', 'A27multiplyB14', 'B11rangemultiplyB14', 'A14multiplyB14', 'A11', 'A10multiplyA9', 'A20rangemultiplyB14', 'A9multiplynumerical_id', 'A14', 'A14multiplyA6', 'A20multiplyB14', 'A25multiplyB14', 'A9', 'B14', 'B10multiplynumerical_id', 'A10', 'A5multiplyB14', 'B14_to_A5_intTarget_0.0_mean', 'A5multiplynumerical_id', 'A12multiplyA15', 'A10multiplyB11range', 'A11multiplynumerical_id', 'B1multiplyB14', 'id', 'B10multiplyB6', 'B6multiplyB7', 'A15', 'A27multiplyB6', 'A24', 'A22multiplyB6', 'id_dispersion', 'B6', 'B14multiplyB6', 'A12', 'A19multiplyB14', 'A10multiplyB11', 'A10multiplynumerical_id', 'B11multiplyB14', 'A11multiplyA6', 'B6multiplynumerical_id', 'A6multiplynumerical_id', 'B14multiplynumerical_id', 'A17multiplyA19', 'B10rangemultiplyB6', 'B9rangemultiplynumerical_id', 'B7', 'A12multiplyA17', 'A10multiplyA25', 'A20rangemultiplynumer

In [13]:
df_predicted=pd.DataFrame({"id":sample_id.values,"predicted":predictions_xgb})
df_predicted.loc[:,"predicted"]=df_predicted["predicted"].apply(lambda x: round(x,3))
df_predicted.to_csv("E:\\jinnan\\predict_result\\jinnan_select_xgb_1_15.csv",header=None,index=False)
df_predicted

,id,predicted
0,sample_1656,0.905
1,sample_1548,0.880
2,sample_769,0.932
3,sample_1881,0.905
4,sample_1807,0.927
5,sample_145,0.927
6,sample_1212,0.935
7,sample_944,0.907
8,sample_829,0.935
9,sample_616,0.927


In [9]:
print("mean_lgb:",np.mean(predictions_lgb))
print("var_lgb:",np.var(predictions_lgb))
print("mean_xgb:",np.mean(predictions_xgb))
print("var_xgb:",np.var(predictions_xgb))

mean_lgb: 0.9255335362943712
var_lgb: 0.0006407562320081167
mean_xgb: 0.9248565008242925
var_xgb: 0.0006447207026118758


In [10]:
from sklearn.linear_model import BayesianRidge

In [11]:
train_stack = np.vstack([oof_lgb,oof_xgb]).transpose()
test_stack = np.vstack([predictions_lgb, predictions_xgb]).transpose()

folds_stack = RepeatedKFold(n_splits=5, n_repeats=2, random_state=4590)
oof_stack = np.zeros(train_stack.shape[0])
predictions = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds_stack.split(train_stack,label_array)):
    print("fold {}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], label_array[trn_idx]
    val_data, val_y = train_stack[val_idx], label_array[val_idx]
    
    clf_3 = BayesianRidge()
    clf_3.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf_3.predict(val_data)
    predictions += clf_3.predict(test_stack) / 10
    
mean_squared_error(label_array, oof_stack)

fold 0
fold 1
fold 2
fold 3
fold 4
fold 5
fold 6
fold 7
fold 8
fold 9


0.00011923631297016711

In [14]:
df_predicted=pd.DataFrame({"id":sample_id.values,"predicted":predictions})
df_predicted.loc[:,"predicted"]=df_predicted["predicted"].apply(lambda x: round(x,3))
df_predicted.to_csv("E:\\jinnan\\predict_result\\jinnan_selected_stacking_1_15.csv",header=None,index=False)
df_predicted

,id,predicted
0,sample_1656,0.905
1,sample_1548,0.881
2,sample_769,0.932
3,sample_1881,0.904
4,sample_1807,0.929
5,sample_145,0.925
6,sample_1212,0.936
7,sample_944,0.903
8,sample_829,0.935
9,sample_616,0.926


In [12]:
print("mean:",np.mean(predictions))
print("var:",np.var(predictions))

mean: 0.9254374729852107
var: 0.000650712951249469
